# `nepare` ASAP `polaris` Competition

This notebook demonstrates using Neural Pairwise Regression (via `nepare`) with the `polaris` benchmarking library to compete in the ASAP discovery competition.

See the `meta` directory in the `nepare` repository for some more theoretical exploration of what this notebook does.

Files in the `utils` subdirectory contain inline comments explaining what the various convenience functions do.

## Requirements
This should work on Python 3.10 or newer - I ran with 3.12

The following dependencies are required:
 - polaris-lib >=0.11.6
 - pandas
 - rdkit
 - lightning
 - torch
 - fastprop
 - mordredcommunity
 - chemprop ~2.1
 - ipywidgets

You will also need to run `pip install .` in the repository's root directory to install `nepare`.

## `polaris` Setup

After running `polaris login` on the command line, we can import everything (checking that the version is recent enough) and then download the benchmark data.

In [22]:
import polaris as po
import pandas as pd

In [23]:
%%capture
competition = po.load_competition("asap-discovery/antiviral-potency-2025")
# or
# competition = po.load_competition("asap-discovery/antiviral-admet-2025")

## Choosing Molecular Embeddings
We can pick a number of different methods for representing molecules as a vector.
For this notebook we will consider just two: `mordred` and ChemProp.
The former uses a vector of ~1,600 molecular descriptors which can be calculated from the graph.
This embedding is _fixed_ or _static_, i.e. it does not change during training.
This has the advantage of packing a lot of chemical knowledge into the representation, but anything that we miss can't be added during training.
ChemProp on the other hand is a _learned_ embedding, which will adapt during training to best regress our target.

There is also the combination of the two - learn a representation during training, and then simply concatenate the molecular descriptors to said representation.
This turned out to work better than either representation on its own for a few of the benchmarks (see the configuration dictionary below), but required some hyperparameter optimization (see `hopt.py`) to get decent results.

In [24]:
EMBEDDING = {
    'HLM': "combined",
    'MLM': "combined",
    'KSOL': "combined",
    'LogD': "combined",
    'MDR1-MDCKII': "combined",
    'pIC50 (MERS-CoV Mpro)': "combined",
    'pIC50 (SARS-CoV-2 Mpro)': "combined",
}

In [25]:
train, test = competition.get_train_test_split()
test_df: pd.DataFrame = test.as_dataframe()
train_df: pd.DataFrame = train.as_dataframe()

In [26]:
train_df

,CXSMILES,HLM,KSOL,MDR1-MDCKII,MLM,LogD
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,NaN,NaN,2.0,NaN,0.3
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,NaN,333.0,0.2,NaN,2.9
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,NaN,NaN,0.5,NaN,0.4
3,NC(=O)[C@H]1CCCN(C(=O)CC2=CC=CC3=C2C=CO3)C1 |&...,NaN,376.0,8.5,NaN,1.0
4,CC1=CC(CC(=O)N2CCC[C@H](C(N)=O)C2)=CC=N1 |&1:11|,NaN,375.0,0.9,NaN,-0.3
...,...,...,...,...,...,...
429,CC(C)NC[C@H](O)COC1=CC=CC2=CC=CC=C12 |&1:5|,25.5,NaN,NaN,63.0,NaN
430,O=C(O)CC1=CC=CC=C1NC1=C(Cl)C=CC=C1Cl,216.0,NaN,NaN,386.0,NaN
431,NCC1=CC(Cl)=CC(C(=O)NC2=CC=C3CNCC3=C2)=C1,NaN,NaN,NaN,NaN,2.0
432,COC(=O)NC1=NC2=CC=C(C(=O)C3=CC=CC=C3)C=C2N1,NaN,NaN,NaN,NaN,2.9


## Train Per-Task Models
`nepare` works by taking the difference in a target value for each input.
Unfortunately this means that when target values are _missing_, we can't simply mask single inputs - we need to mask entire pairs.
Rather than do this and end up with a really sparse training set, we will just train one model for each target property.

This is OK in terms of the amount of training data we have - we might drop 25% of the data, but we end up squaring the amount of remaining data, nullifying the issue.

In [27]:
from pathlib import Path

import torch
import lightning
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from chemprop.nn.agg import MeanAggregation
from chemprop.nn.message_passing import BondMessagePassing
from fastprop.data import standard_scale, inverse_standard_scale

from nepare.data import PairwiseAugmentedDataset, PairwiseAnchoredDataset, PairwiseInferenceDataset
from nepare.nn import LearnedEmbeddingNeuralPairwiseRegressor, NeuralPairwiseRegressor, FeedforwardNeuralNetwork
from nepare.inference import predict

from utils.splitting import split
from utils.chemprop_wrappers import smiles2molgraphcache, collate_fn, ChemPropEmbedder
from utils.mordred_wrappers import smi2features
from utils.metrics import evaluate_predictions
from utils.combined_embeddings import MordredChemPropEmbedder, collate_fn_combined, ArbitrarilyMoreComplicatedEmbedder

This huge block of code is responsible for training each model, reporting its performance, and running inference on the test data.
Check the inline comments for more explanation.

In [28]:
predictions = {}  # for polaris
val_score = []  # for my own checking of model performance
output_dir = Path("lightning_logs")
tasks = list(competition.target_cols)
for task_n, task_name in enumerate(tasks):
    # copy from the train dataframe each time to avoid accidentally messing up all the training data
    task_df = train_df[["CXSMILES", task_name]].copy()
    task_df.dropna(inplace=True)
    task_df.reset_index(inplace=True)

    # this function splits the data into a training set (for setting weights) and validation set (for early stopping)
    # the validation set will be equal in size to the test dataframe and composed of 50% randomly selected molecules
    # and the remainder molecular that are highly similar to the test molecules
    #
    # update: first approach wasn't great - try fitting on only the validation set from before to avoid overfitting
    # and keep it small even then
    _, val_idxs = split(task_df["CXSMILES"], test_df["CXSMILES"])
    if len(val_idxs) > 150:
        val_idxs = val_idxs[:150]
    train_idxs = val_idxs[5:]
    val_idxs = val_idxs[:5]

    # rescale our target values, both for metric reporting and NN performance
    train_targets = torch.tensor(task_df[task_name].iloc[train_idxs].to_numpy(), dtype=torch.float32).reshape(-1, 1)  # 2d!
    train_targets, target_means, target_vars = standard_scale(train_targets)
    val_targets = torch.tensor(task_df[task_name].iloc[val_idxs].to_numpy(), dtype=torch.float32).reshape(-1, 1)  # 2d!
    val_targets = standard_scale(val_targets, target_means, target_vars)

    # avoid some code duplication
    dataset_kwargs = dict(batch_size=256)

    # each embedding does fundamentally the same thing:
    # - turn the molecules into tensors
    # - pack those tensors into a nepare dataset that auto-magically augments it
    # - initializes the model
    #
    # the big difference is that embeddings using ChemProp have a special function for collating
    # their batches and also must instantiate an additional module used to learn an embedding
    if EMBEDDING[task_name] == "mordred":
        Model = NeuralPairwiseRegressor
        train_features, feature_means, feature_vars = smi2features(task_df["CXSMILES"][train_idxs])
        val_features, _, _ = smi2features(task_df["CXSMILES"][val_idxs], feature_means, feature_vars)
        test_features, _, _ = smi2features(test_df["CXSMILES"], feature_means, feature_vars)
        # setup the datasets
        train_dataset = PairwiseAugmentedDataset(train_features, train_targets, how='full')
        val_dataset = PairwiseAnchoredDataset(train_features, train_targets, val_features, val_targets, how='full')
        val_absolute_dataset = PairwiseInferenceDataset(train_features, train_targets, val_features, how='full')
        test_dataset = PairwiseInferenceDataset(train_features, train_targets, test_features, how='full')
        # build the model
        npr = Model(train_features.shape[1], 70, 3)
    elif EMBEDDING[task_name] == "chemprop":
        Model = LearnedEmbeddingNeuralPairwiseRegressor
        # featurize the data
        train_mgc = smiles2molgraphcache(task_df["CXSMILES"][train_idxs])
        val_mgc = smiles2molgraphcache(task_df["CXSMILES"][val_idxs])
        test_mgc = smiles2molgraphcache(test_df["CXSMILES"])
        # setup the datasets
        dataset_kwargs["collate_fn"] = collate_fn
        train_dataset = PairwiseAugmentedDataset(train_mgc, train_targets, how='sut')
        val_dataset = PairwiseAnchoredDataset(train_mgc, train_targets, val_mgc, val_targets, how='half')
        val_absolute_dataset = PairwiseInferenceDataset(train_mgc, train_targets, val_mgc, how='half')
        test_dataset = PairwiseInferenceDataset(train_mgc, train_targets, test_mgc, how='half')
        # build the model
        mp = BondMessagePassing(d_h=400)
        agg = MeanAggregation()
        embedder = ChemPropEmbedder(mp, agg)
        npr = Model(embedder, 400, 200, 2, lr=5e-5)
    elif EMBEDDING[task_name] == "combined":
        Model = LearnedEmbeddingNeuralPairwiseRegressor
        # featurize the data
        train_features, feature_means, feature_vars = smi2features(task_df["CXSMILES"][train_idxs])
        val_features, _, _ = smi2features(task_df["CXSMILES"][val_idxs], feature_means, feature_vars)
        test_features, _, _ = smi2features(test_df["CXSMILES"], feature_means, feature_vars)
        train_mgc = smiles2molgraphcache(task_df["CXSMILES"][train_idxs])
        val_mgc = smiles2molgraphcache(task_df["CXSMILES"][val_idxs])
        test_mgc = smiles2molgraphcache(test_df["CXSMILES"])
        train_tuples = [(train_mgc[i], train_features[i, :]) for i in range(len(train_targets))]
        val_tuples = [(val_mgc[i], val_features[i, :]) for i in range(len(val_targets))]
        test_tuples = [(test_mgc[i], test_features[i, :]) for i in range(len(test_mgc))]
        # setup the datasets
        train_dataset = PairwiseAugmentedDataset(train_tuples, train_targets, how='sut')
        val_dataset = PairwiseAnchoredDataset(train_tuples, train_targets, val_tuples, val_targets, how='half')
        val_absolute_dataset = PairwiseInferenceDataset(train_tuples, train_targets, val_tuples, how='half')
        test_dataset = PairwiseInferenceDataset(train_tuples, train_targets, test_tuples, how='half')
        dataset_kwargs["collate_fn"] = collate_fn_combined
        # build the model
        _size = 84
        _layers = 2
        mp = BondMessagePassing(d_h=_size, depth=_layers)
        agg = MeanAggregation()
        embedder = ArbitrarilyMoreComplicatedEmbedder(mp, agg, _size, _layers)
        npr = LearnedEmbeddingNeuralPairwiseRegressor(embedder, 2*_size, 2*_size, _layers, lr=1e-5)
        # npr = Model(embedder, mp.output_dim + train_features.shape[1], 200, 2, lr=1e-5)


    # classic lightning training, inference
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, **dataset_kwargs)
    val_loader = torch.utils.data.DataLoader(val_dataset, **dataset_kwargs)
    val_absolute_loader = torch.utils.data.DataLoader(val_absolute_dataset, **dataset_kwargs)
    predict_loader = torch.utils.data.DataLoader(test_dataset, **dataset_kwargs)
    early_stopping = EarlyStopping(monitor="validation/loss", patience=10)
    name = "_".join([EMBEDDING[task_name], task_name])
    model_checkpoint = ModelCheckpoint(dirpath=output_dir / name, monitor="validation/loss")
    logger = TensorBoardLogger(save_dir=output_dir, name=name, default_hp_metric=False)
    trainer = lightning.Trainer(max_epochs=100, log_every_n_steps=1, callbacks=[early_stopping, model_checkpoint], logger=logger)
    trainer.fit(npr, train_loader, val_loader)
    npr = Model.load_from_checkpoint(model_checkpoint.best_model_path)

    # we use the predict function from nepare, which automatically maps difference-space predictions back to absolute space
    y_pred, y_stdev = predict(npr, val_absolute_loader)

    # checking our performance on the validation set
    # leave in the scaled space so that the average is weighted equally among the targets
    results_dict = evaluate_predictions(val_targets.flatten().numpy(), y_pred.flatten().numpy())
    val_score.append((len(val_targets), results_dict["MAE"]))
    # now go back to correct scale
    y_pred = inverse_standard_scale(y_pred, target_means, target_vars)
    y_true = inverse_standard_scale(val_targets, target_means, target_vars)
    print(f"{task_name=}:\n - {"\n - ".join([f'{name}: {score:.4f}' for name, score in evaluate_predictions(y_true.flatten().numpy(), y_pred.flatten().numpy()).items()])}")

    # finally predict on the test set, descale, and save them for later
    y_pred, y_stdev = predict(npr, predict_loader)
    y_pred = inverse_standard_scale(y_pred, target_means, target_vars)
    predictions[task_name] = y_pred.flatten().tolist()

/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'embedding_module' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['embedding_module

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


task_name='LogD':
 - Pearson r: 0.9812
 - MSE: 0.0298
 - MAE: 0.1665


Predicting: |          | 0/? [00:00<?, ?it/s]

The hope is that the below number roughly matches with the actual performance on the `polaris` leaderboard.
I say _hope_ because the composition of the blind test set and my validation set will invariably be different, just hopefully not _so_ different that the validation set actually causes _bad_ early stopping (i.e. too early or too late).

In [31]:
print(f"Validation Scaled Weighted Average MAE {sum(n*s for n, s in val_score) / sum(n for n, _ in val_score):.4f}")

Validation Scaled Weighted Average MAE 0.2660


This last block is commented out because it will fail (unless you are me) - you can replace the inputs with your own if you are submitting this for yourself.

In [30]:
# competition.submit_predictions(
#     predictions=predictions,
#     prediction_name="nepare",
#     prediction_owner="jacksonburns",
#     report_url="https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/meta",
#     github_url = "https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/notebooks/polaris_asap/main.ipynb",
#     description = "Neural Pairwise Regression with ChemProp + Mordred",
# )